In [1]:
from src.llm_plan.Agent import AgentCoder
from src.llm_plan.LLM import GPT_4o

model = GPT_4o()

# A description of blocks world
task_description = """The Door and Key Problem
The Problem: You are in a room with a locked door and a key. To escape, you must open the door.

The Goal: The door must be in the "open" state.

The Constraints & Actions:

Pickup Key: You can pick up the key.

Unlock Door: You can only unlock the door if you are holding the key. The door then becomes open."""

agent_coder = AgentCoder(model, "SingleAgentDoorAndKey", task_description)

In [2]:
agent_coder.code_task(history="", task="problem")

In [ ]:
agent_coder.code_task(history="<HISTORY>", task="environment")

In [ ]:
agent_coder.code_task(history="<HISTORY>", task="experiment")